In [1]:
import pandas as pd
import numpy as np
import re

#read in excel raw data for all conditions
fname = "/Users/leemc/Documents/GitHub/blood-vessel-diameter/Data/THA-auc-all-lines.xlsx"
auc_df = pd.read_excel(fname)

auc_df

,Unnamed: 0,011017_12green_LineProfile_2.tif,011017_12green_LineProfile_3.tif,011017_12 green_LineProfile_4.tif,011017_12 green_LineProfile_5.tif,011017_12 green_LineProfile_7.tif,011017_13 green_LineProfile_3.tif,011017_13 green_LineProfile_4.tif,011017_13 green_LineProfile_5.tif,011017_13 green_LineProfile_6.tif,...,031618_24 green v1_LineProfile_3.tif,031618_24 green v1_LineProfile_4.tif,031618_24 green v1_LineProfile_5.tif,031618_24 green_LineProfile_1.tif,031618_24 green_LineProfile_2.tif,031618_24 green_LineProfile_3.tif,031618_24 green_LineProfile_4.tif,031618_24 green_LineProfile_5.tif,031618_24 green_LineProfile_6.tif,031618_24 green_LineProfile_7.tif
0,Baseline,0.00000,0.000,0.00000,0.0000,0.0000,0.000,0.0000,0.000,0.000,...,0.00,0.000,0.000,0.000000,0.0000,0.0000,0.00000,0.0000,0.0000,0.000
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Total Area,6.08700,1.814,11.42000,1.2800,1.1640,2.343,2.0640,1.638,3.061,...,13.72,8.844,2.989,7.711000,6.3150,3.3970,6.67300,1.5460,1.2570,1.798
3,Total Peak Area,0.01965,1.484,0.01944,0.8601,0.3432,1.705,0.9162,0.345,2.475,...,13.27,8.563,2.304,0.001317,0.0208,0.0656,0.08072,0.3139,0.2347,1.263
4,Number of Peaks,3.00000,12.000,4.00000,18.0000,19.0000,11.000,15.0000,18.000,7.000,...,1.00,1.000,9.000,1.000000,4.0000,2.0000,1.00000,6.0000,7.0000,8.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,Last X,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
201,Peak X,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
202,Peak Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
203,Area,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
#Transpose dataframes so that the lines are the rows and the measurements are columns 
auc_trans = auc_df.set_index('Unnamed: 0').transpose()
auc_trans

#Drop unnecessary columns
auc_PA = auc_trans['%Area']
num_peaks = auc_trans['Number of Peaks']


#Find the largest % area peak in each line
peaks = np.arange(1, num_peaks.max()+1, 1).astype(int).tolist()
auc_PA.columns = peaks
auc_PA.fillna(value = 0, inplace = True)
auc_PA['Max % Area'] = auc_PA.max(axis=1)
auc_PA['Max Peak'] = auc_PA.idxmax(axis=1).astype(int)

auc_PA


/Users/leemc/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4147: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(
<ipython-input-2-8cc147691681>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  auc_PA['Max % Area'] = auc_PA.max(axis=1)
<ipython-input-2-8cc147691681>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ret

,1,2,3,4,5,6,7,8,9,10,...,18,19,20,21,22,23,24,25,Max % Area,Max Peak
011017_12green_LineProfile_2.tif,52.070,39.7400,8.1910,0.0000,0.0000,0.00000,0.00000,0.000,0.000,0.000000,...,0.00000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,52.07,1
011017_12green_LineProfile_3.tif,1.706,3.3590,26.3000,31.7800,6.3960,6.53900,7.94500,2.799,6.181,0.003334,...,0.00000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,31.78,4
011017_12 green_LineProfile_4.tif,34.350,0.5774,64.9600,0.1189,0.0000,0.00000,0.00000,0.000,0.000,0.000000,...,0.00000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,64.96,3
011017_12 green_LineProfile_5.tif,9.865,4.8460,0.2284,9.0410,0.8869,10.04000,17.84000,0.877,7.420,13.600000,...,0.07186,0.000,0.0,0.0,0.0,0.0,0.0,0.0,17.84,7
011017_12 green_LineProfile_7.tif,3.635,2.2180,13.5000,2.5910,4.4300,0.03434,0.09107,11.410,6.360,4.813000,...,25.12000,3.719,0.0,0.0,0.0,0.0,0.0,0.0,25.12,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
031618_24 green_LineProfile_3.tif,87.260,12.7400,0.0000,0.0000,0.0000,0.00000,0.00000,0.000,0.000,0.000000,...,0.00000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,87.26,1
031618_24 green_LineProfile_4.tif,100.000,0.0000,0.0000,0.0000,0.0000,0.00000,0.00000,0.000,0.000,0.000000,...,0.00000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,100.00,1
031618_24 green_LineProfile_5.tif,92.100,0.3839,0.1691,2.4310,0.1718,4.74700,0.00000,0.000,0.000,0.000000,...,0.00000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,92.10,1
031618_24 green_LineProfile_6.tif,72.210,1.2690,1.3870,0.1935,4.3530,5.29400,15.30000,0.000,0.000,0.000000,...,0.00000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,72.21,1


In [3]:
#Test if peak largest % area peak and largest area peak are the same
auc_area = auc_trans['Area']
auc_area.columns = peaks
auc_area.fillna(value = 0, inplace = True)
auc_area['Max Area'] = auc_area.max(axis=1)
auc_area['Max Peak'] = auc_area.idxmax(axis=1).astype(int)

test = auc_PA['Max Peak'] == auc_area['Max Peak']
False in test

/Users/leemc/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4147: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(
<ipython-input-3-80a69c234002>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  auc_area['Max Area'] = auc_area.max(axis=1)
<ipython-input-3-80a69c234002>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ret

False

In [3]:
#Add max peaks to the dataframes with the rest of the data
auc_peaks = auc_trans.drop(columns=['Baseline', 'Total Area', 'Total Peak Area', 'Number of Peaks']).dropna(axis = 'columns', how = 'all')
#auc_peaks['Max % Area'] = auc_PA['Max % Area']
auc_peaks['Max Peak'] = auc_PA['Max Peak']
auc_peaks

Unnamed: 0,First X,Last X,Peak X,Peak Y,Area,%Area,First X,Last X,Peak X,Peak Y,...,Peak Y,Area,%Area,First X,Last X,Peak X,Peak Y,Area,%Area,Max Peak
011017_12green_LineProfile_2.tif,1.823,3.048,2.0,0.01311,0.010230,52.070,70.900,72.030,71.0,0.010370,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
011017_12green_LineProfile_3.tif,1.000,1.991,1.0,0.05107,0.025310,1.706,3.266,5.821,4.0,0.040210,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4
011017_12 green_LineProfile_4.tif,2.624,3.144,3.0,0.02567,0.006678,34.350,81.780,82.010,82.0,0.000986,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
011017_12 green_LineProfile_5.tif,1.000,5.841,5.0,0.03528,0.084850,9.865,6.202,9.669,7.0,0.026360,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7
011017_12 green_LineProfile_7.tif,1.871,3.497,3.0,0.01459,0.012480,3.635,4.533,5.710,5.0,0.012950,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
031618_24 green_LineProfile_3.tif,4.160,10.710,6.0,0.01801,0.057240,87.260,11.250,13.060,12.0,0.009261,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
031618_24 green_LineProfile_4.tif,6.469,11.810,8.0,0.02340,0.080720,100.000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
031618_24 green_LineProfile_5.tif,5.653,16.660,7.0,0.05491,0.289000,92.100,20.630,21.210,21.0,0.004143,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
031618_24 green_LineProfile_6.tif,5.762,18.060,8.0,0.02516,0.169400,72.210,72.980,74.540,74.0,0.003664,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [4]:
#Deal with duplicate column names
peak_str = [str(i) for i in peaks]
peak_str
f = ['First X_' + sub for sub in peak_str]
l = ['Last X_' + sub for sub in peak_str]
px = ['Peak X_' + sub for sub in peak_str]
py = ['Peak Y_' + sub for sub in peak_str]
a = ['Area_' + sub for sub in peak_str]
pa = ['%Area_' + sub for sub in peak_str]

f_col = {'First X' : f}
l_col = {'Last X' : l}
px_col = {'Peak X' : px}
py_col = {'Peak Y' : py}
a_col = {'Area' : a}
pa_col = {'%Area' : pa}

auc_peaks.rename(columns=lambda c: f_col[c].pop(0) if c in f_col.keys() else c, inplace=True)
auc_peaks.rename(columns=lambda c: l_col[c].pop(0) if c in l_col.keys() else c, inplace=True)
auc_peaks.rename(columns=lambda c: px_col[c].pop(0) if c in px_col.keys() else c, inplace=True)
auc_peaks.rename(columns=lambda c: py_col[c].pop(0) if c in py_col.keys() else c, inplace=True)
auc_peaks.rename(columns=lambda c: a_col[c].pop(0) if c in a_col.keys() else c, inplace=True)
auc_peaks.rename(columns=lambda c: pa_col[c].pop(0) if c in pa_col.keys() else c, inplace=True)

auc_peaks

Unnamed: 0,First X_1,Last X_1,Peak X_1,Peak Y_1,Area_1,%Area_1,First X_2,Last X_2,Peak X_2,Peak Y_2,...,Peak Y_24,Area_24,%Area_24,First X_25,Last X_25,Peak X_25,Peak Y_25,Area_25,%Area_25,Max Peak
011017_12green_LineProfile_2.tif,1.823,3.048,2.0,0.01311,0.010230,52.070,70.900,72.030,71.0,0.010370,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
011017_12green_LineProfile_3.tif,1.000,1.991,1.0,0.05107,0.025310,1.706,3.266,5.821,4.0,0.040210,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4
011017_12 green_LineProfile_4.tif,2.624,3.144,3.0,0.02567,0.006678,34.350,81.780,82.010,82.0,0.000986,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
011017_12 green_LineProfile_5.tif,1.000,5.841,5.0,0.03528,0.084850,9.865,6.202,9.669,7.0,0.026360,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7
011017_12 green_LineProfile_7.tif,1.871,3.497,3.0,0.01459,0.012480,3.635,4.533,5.710,5.0,0.012950,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
031618_24 green_LineProfile_3.tif,4.160,10.710,6.0,0.01801,0.057240,87.260,11.250,13.060,12.0,0.009261,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
031618_24 green_LineProfile_4.tif,6.469,11.810,8.0,0.02340,0.080720,100.000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
031618_24 green_LineProfile_5.tif,5.653,16.660,7.0,0.05491,0.289000,92.100,20.630,21.210,21.0,0.004143,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
031618_24 green_LineProfile_6.tif,5.762,18.060,8.0,0.02516,0.169400,72.210,72.980,74.540,74.0,0.003664,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [5]:
#Cycle through lines to get a dataframe with only max peak data, searching for numbers that match max peak
c = -1
new_columns = ['First X', 'Last X', 'Peak X', 'Peak Y', 'Area', '%Area', 'Peak']
max_peak_df = pd.DataFrame(columns = new_columns)
for p in auc_peaks['Max Peak']:
    m = []
    c += 1
    line = auc_peaks.index[c]
    for i in auc_peaks.columns:
        match = re.findall('.*_' + str(p) + '$', i)
        match_to_string = ''.join(map(str, match))
        if not match_to_string:
            pass
        else:
            m.append(match_to_string)
    line_max_peak = auc_peaks.loc[line, m]
    max_peak_df_small = pd.DataFrame(line_max_peak).transpose()
    max_peak_df_small['Peak'] = p
    max_peak_df_small.columns = new_columns
    max_peak_df = max_peak_df.append(max_peak_df_small)
max_peak_df
#max_peak_df.to_csv('THA largest percent area peak.csv')

,First X,Last X,Peak X,Peak Y,Area,%Area,Peak
011017_12green_LineProfile_2.tif,1.823,3.048,2.0,0.01311,0.01023,52.07,1
011017_12green_LineProfile_3.tif,24.180,39.480,33.0,0.05812,0.47150,31.78,4
011017_12 green_LineProfile_4.tif,83.900,85.080,84.0,0.01423,0.01263,64.96,3
011017_12 green_LineProfile_5.tif,27.740,36.350,31.0,0.03029,0.15340,17.84,7
011017_12 green_LineProfile_7.tif,76.490,82.300,77.0,0.02586,0.08623,25.12,18
...,...,...,...,...,...,...,...
031618_24 green_LineProfile_3.tif,4.160,10.710,6.0,0.01801,0.05724,87.26,1
031618_24 green_LineProfile_4.tif,6.469,11.810,8.0,0.02340,0.08072,100.00,1
031618_24 green_LineProfile_5.tif,5.653,16.660,7.0,0.05491,0.28900,92.10,1
031618_24 green_LineProfile_6.tif,5.762,18.060,8.0,0.02516,0.16940,72.21,1
